In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
# Step 1: Load the dataset
file_path = "10_Property_stolen_and_recovered.xlsx"
xls = pd.ExcelFile(file_path)
# View available sheets
print("Available Sheets:", xls.sheet_names)
# Load the relevant sheet (e.g., first sheet)
df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
# Step 2: Basic Cleaning
df.dropna(how='all', inplace=True)  # remove completely empty rows
df.columns = [col.strip() for col in df.columns]  # clean column names
df = df[df['Year'].notnull()]  # keep rows with valid year values
# Display first few rows
print(df.head())
# Step 3: Exploratory Data Analysis
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='Year', y='Property Stolen (₹)', hue='State/UT')
plt.title("Property Stolen Over Years")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
# Step 4: Add Recovery Rate column (if not present)
if 'Recovery Rate (%)' not in df.columns:
    df['Recovery Rate (%)'] = (df['Property Recovered (₹)'] / df['Property Stolen (₹)']) * 100
# Step 5: Define a Gradio UI for visualization
def recovery_query(state: str, year: int):
    data = df[(df['State/UT'] == state) & (df['Year'] == year)]
    if not data.empty:
        stolen = float(data['Property Stolen (₹)'].values[0])
        recovered = float(data['Property Recovered (₹)'].values[0])
        rate = float(data['Recovery Rate (%)'].values[0])
        return f"Stolen: ₹{stolen:.2f}\nRecovered: ₹{recovered:.2f}\nRecovery Rate: {rate:.2f}%"
    else:
        return "No data found for the selected state and year."
states = sorted(df['State/UT'].unique())
years = sorted(df['Year'].unique())
gr.Interface(
    fn=recovery_query,
    inputs=[gr.Dropdown(choices=states, label="State/UT"),
            gr.Dropdown(choices=years, label="Year")],
    outputs="text",
    title="Property Recovery Stats",
    description="Select a State/UT and Year to view property stolen, recovered, and recovery rate."
).launch()